# ПОРТФЕЛИ МАРКОВИЦА И ТОБИНА


## Загрузка библиотек

In [84]:
from scipy.optimize import linprog, minimize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly_express as px
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Загрузка базы данных с котировками акций

In [85]:
df = pd.read_excel('database.xlsx')
df = df.set_index('DATE')
df.head(10)

,MTSS,SBER,AFLT
DATE,,,
2022-04-18,188.05,116.97,30.62
2022-04-25,209.05,128.80,31.14
2022-05-02,211.00,123.10,30.10
2022-05-09,200.85,120.20,29.26
2022-05-16,238.50,122.20,29.32
2022-05-23,246.90,121.22,28.54
2022-05-30,259.50,119.21,26.74
2022-06-06,274.00,118.07,26.30
2022-06-13,274.70,123.88,27.52


## Динамика стоимости акций

In [86]:
px.line(df/df.iloc[0])

## Динамика доходности акций

In [87]:
returns = df.pct_change()
px.line(returns)

## Портфель Марковица максимальной доходности

Составим портфель Марковица максимальной доходности. Сначала не будем использовать дополнительное условие на рис. Используя функцию __linprog__ из библиотеки scipy найдем максимальное значение целевой функции симплекс-методом. 

$$max\sum_i^N{x_im_i}$$
$$\sum_i^N{x_i} = 1,$$    
$$x_i \ge 0, i = 1,...,N$$

In [88]:
def MaximizeReturns(returns):

    expected_return = returns.mean()
    cov = returns.cov()
    n = len(expected_return)

    c = np.matrix(-1 * np.array(expected_return))
    A_eq = np.matrix(np.ones(n))
    b_eq = 1.
    opt = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=(0, 1), method='simplex')

    x = opt.x
    r_p = x.dot(expected_return.T)
    sigm_p = np.sqrt(np.dot(np.dot(x, cov), x.T))

    return x, r_p, sigm_p

In [89]:
x, r_p, sigm_p = MaximizeReturns(returns)
index = np.arange(1, len(x) + 1)

data = {'STOCK' : df.columns, 
        'E(R)' : returns.mean(), 
        'sigm(R)' : returns.std(),
        'WEIGHTS': x}

parametrs = pd.DataFrame.from_dict(data).set_index(index)

fig = px.pie(parametrs, values='WEIGHTS', names='STOCK', title='Портфель максимальной доходности')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print(f'Доходность: {round(r_p * 100, 2)}%\nРиск: {round(sigm_p * 100, 2)}%')
parametrs

Доходность: 1.18%
Риск: 4.47%


,STOCK,E(R),sigm(R),WEIGHTS
1,MTSS,0.005412,0.045689,0.0
2,SBER,0.011752,0.044722,1.0
3,AFLT,0.003608,0.040234,0.0


## Портфель Марковица минимального риска

In [153]:
def kramers_matrix(returns, min_r=None):

    cov = returns.cov()
    e = np.matrix(returns.mean())
    ones = np.matrix(np.ones(len(cov)))

    if min_r is None:
        last_row = np.linspace(0, 0, len(cov) + 1)
        last_row[:3] = 1 
        last_row = np.matrix(last_row)
        A = np.hstack((cov, ones.T))
        A = np.vstack((A, last_row))
    
    else:
        e_row = np.linspace(0, 0, len(cov) + 2)
        e_row[:3] = returns.mean()
        last_row = np.linspace(0, 0, len(cov) + 2)
        last_row[:3] = 1
        last_row[3] = min_r
        last_row = np.matrix(last_row)
        A = np.hstack((cov, e.T, ones.T))
        A = np.vstack((A, e_row, last_row))

    return A

def MinimizeRisk(returns):
    
    def kramer(A, b):

     d = np.linalg.det(A)
     x = []
     for i in range(len(b)):
        a = np.copy(A)
        a[:, i] = b
        k = np.linalg.det(a)
        x.append(k/d)

     return x

    cov = np.array(returns.cov())
    e = returns.mean()

    A = kramers_matrix(returns)

    b = [0, 0, 0, 1]

    x = np.array(kramer(A, b))[:3]
    r_p = x.dot(e.T)
    sigm_p = np.sqrt(np.dot(np.dot(x, cov), x.T))

    return x, r_p, sigm_p

In [154]:
x, r_p, sigm_p = MinimizeRisk(returns)

parametrs.iloc[:, 1] = returns.mean()
parametrs.iloc[:, 2] = returns.std()
parametrs.iloc[:, 3] = x

fig = px.pie(parametrs, values='WEIGHTS', names='STOCK', title='Портфель минимального риска')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print(f'Доходность: {round(r_p * 100, 2)}%\nРиск: {round(sigm_p * 100, 2)}%')
parametrs

Доходность: 0.56%
Риск: 3.38%


,STOCK,E(R),sigm(R),WEIGHTS
1,MTSS,0.005412,0.045689,0.368630
2,SBER,0.011752,0.044722,0.159393
3,AFLT,0.003608,0.040234,0.471976


In [155]:
def MinimizeRiskConstr(returns, min_r):
    
    def kramer(A, b):

     d = np.linalg.det(A)
     x = []
     for i in range(len(b)):
        a = np.copy(A)
        a[:, i] = b
        k = np.linalg.det(a)
        x.append(k/d)

     return x

    cov = np.array(returns.cov())
    e = returns.mean()

    A = kramers_matrix(returns, min_r)

    b = [0, 0, 0, min_r, 1]

    x = np.array(kramer(A, b))[:3]
    r_p = x.dot(e.T)
    sigm_p = np.sqrt(np.dot(np.dot(x, cov), x.T))

    return x, r_p, sigm_p

In [156]:
x, r_p, sigm_p = MinimizeRiskConstr(returns, 0.01)

parametrs.iloc[:, 1] = returns.mean()
parametrs.iloc[:, 2] = returns.std()
parametrs.iloc[:, 3] = x

fig = px.pie(parametrs, values='WEIGHTS', names='STOCK', title='Портфель минимального риска')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print(f'Доходность: {round(r_p * 100, 2)}%\nРиск: {round(sigm_p * 100, 2)}%')
parametrs

Доходность: 1.0%
Риск: 3.89%


,STOCK,E(R),sigm(R),WEIGHTS
1,MTSS,0.005412,0.045689,0.274139
2,SBER,0.011752,0.044722,0.723813
3,AFLT,0.003608,0.040234,0.002888
